In [2]:
from transformers import AutoTokenizer, AutoModel
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
import torch
from torch import nn

import pandas as pd

In [3]:
model = AutoModel.from_pretrained("csebuetnlp/banglabert")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")

In [4]:
original_sentence = "আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু"
fake_sentence = "আপনি হতাশ কারণ  আমার জন্য অনেক কিছু করেছেন।"
fake_sentence = normalize(fake_sentence) # this normalization step is required before tokenizing the text
original_sentence = normalize(original_sentence)

In [5]:
fake_tokens = tokenizer.tokenize(fake_sentence)
fake_inputs = tokenizer.encode_plus(fake_sentence, return_tensors="pt")
# fake_inputs

original_inputs = tokenizer.encode_plus(original_sentence, return_tensors="pt")

In [5]:
discriminator_outputs = model(fake_inputs['input_ids'],attention_mask= fake_inputs['attention_mask'])#.logits
discriminator_outputs2 = model(original_inputs['input_ids'],attention_mask= original_inputs['attention_mask'])#.logits
# predictions = torch.round((torch.sign(discriminator_outputs) + 1) / 2)

In [6]:
input1 = discriminator_outputs['last_hidden_state'][:,0,:]
input2 = discriminator_outputs2['last_hidden_state'][:,0,:]

In [7]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

In [8]:
output = cos(input1, input2)
output

tensor([0.8526], grad_fn=<SumBackward1>)

In [6]:
df_train = pd.read_csv("dataset/bornon_train_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)
df_test = pd.read_csv("dataset/bornon_test_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)

/tmp/ipykernel_19781/2331687903.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_train = pd.read_csv("dataset/bornon_train_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/bornon_train_token.txt'

In [10]:
def consolidate(gdf):
    captions = gdf['caption'].to_list()
    return captions

df1 = df_train.groupby("image_name").apply(consolidate, include_groups=False).reset_index().rename(columns={0: 'captions'})
df2 = df_test.groupby("image_name").apply(consolidate, include_groups=False).reset_index().rename(columns={0: 'captions'})

In [11]:
df1.sort_values(by='image_name', key=lambda x: pd.to_numeric(x.str.rstrip('.jpg'), errors='coerce'), inplace=True)
df2.sort_values(by='image_name', key=lambda x: pd.to_numeric(x.str.rstrip('.jpg'), errors='coerce'), inplace=True)

df = pd.concat([df1, df2], axis=0)
df = df.reset_index()

In [16]:
def top_3_captions(captions: list):
    cap_discriminator_outputs = []
    cap_inputs = []
    for i in range(len(captions)):
        normalised = normalize(captions[i])
        cap_inputs.append(tokenizer.encode_plus(normalised, return_tensors="pt"))
        
    for i in range(len(cap_inputs)):
        cap_discriminator_outputs.append(model(cap_inputs[i]['input_ids'],attention_mask= cap_inputs[i]['attention_mask'])['last_hidden_state'][:,0,:])
        
    return cap_discriminator_outputs
    

In [18]:
for idx, row in df.iterrows():
    # print(idx, row['image_name'], row['captions'])
    outs = top_3_captions(captions=row['captions'])
    print(outs)
    break

[tensor([[-1.6503e-02, -3.9037e-02,  1.3451e-01, -1.3305e-01, -1.7386e-01,
         -3.6409e-01, -7.2003e-02,  5.3091e-01, -2.6134e-01, -1.9454e-01,
          6.4820e-01,  3.3451e-01, -8.0517e-02,  4.4214e-01, -1.9171e-01,
          5.9826e-01, -4.6228e-01, -3.4849e-01,  2.1259e-01, -2.7094e-01,
         -5.6633e-01, -2.9738e-01, -7.5242e-02, -1.7270e-02, -7.9227e-01,
          2.7233e-02,  7.5879e-01, -3.5004e-02, -3.4467e-01, -3.3944e-01,
          3.3403e-01,  4.6115e-01, -9.2491e-01,  1.1881e-01,  4.4296e-03,
          1.3809e-01,  9.9439e-02, -1.3646e-01,  1.2569e-01,  3.8817e-01,
         -1.4834e-01,  3.9538e-01,  6.5963e-02, -2.6531e-01,  2.5619e-01,
          1.7289e-01,  2.7641e-01, -1.2617e-01, -1.9667e+00,  3.1484e-01,
          4.7363e-01, -5.2311e-01, -3.6157e-01, -4.8936e-01,  1.5949e-01,
          1.8223e-01,  1.1703e-01,  4.5596e-01,  5.3831e-01, -6.7783e-01,
         -3.1728e-01, -5.1567e-01, -8.5199e-01, -3.0553e-02, -5.8214e-01,
          5.0426e-01, -3.8020e-01,  2

In [19]:
outs[0].shape

torch.Size([1, 768])